# Parameters for simulations
This notebook allows to create a config.json file containing all the parameters for the simulation to be passed to the main script scripts/run_simulations.py

In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd

In [2]:
# System configuration: edit name of the dynamical system and its parameters
from dyn_net.utils.criticality import find_theta_c_from_degree_distribution

# Degree distribution used by the configuration model

degree_distribution = {
    "name": "poisson",
    "params": {
        "lambda": 5,
    },
}


system = {
    "name": "double_well_network",
    "params": {
        "theta": 1.0,
        "alpha_rot": 0.0,
    },
}

# Network configuration: edit name, size = number of particles and parameters of the network
network = {
    "name": "configuration_model",
    "params": {
        "n": 1000,
        "degree_distribution": degree_distribution,
    },
}

# Noise configuration
sigma = 0.4
noise = {
    "name": "additive_gaussian",
    "params": {
        "sigma": sigma,
    },
}

# Integrator configuration
integrator = {
    "tmin": 0.0,
    "tmax": 5000.0,
    "dt": 0.01,
    "stats_every": 100, # every "stats_every" timesteps get the appropriate statistics and write them to file
    "state_every": 1000, # every "state_every" timesteps write the state of the system to file
    "write_stats_at_start": True,
    "write_state_at_start": True,
}

# Initial condition configuration
initial_condition = {
    "type": "normal", # choose the type of initial condition + its parameters
    "std": 1.0,
}

config = {
    "system": system,
    "network": network,
    "noise": noise,
    "integrator": integrator,
    "initial_condition": initial_condition,
}


In [3]:
# The json file will be saved at root_folder/path_folder/name_config_file
path_folder = "configs"
name_config_file = "config_double_well_configuration_model_poisson.json"

# Save it in the configs folder
repo_root = Path.cwd().parent
output_path = repo_root / path_folder / name_config_file
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(json.dumps(config, indent=2))
print(f"Wrote config to {output_path}")


Wrote config to /Users/niccolo/Desktop/Dynamics_Networks/configs/config_double_well_configuration_model_poisson.json


### Create a sweep table (for phase diagrams)
When one parameter gets changed we can use the config file as a base and then override the value from a sweep table!
This cell writes a simple TSV file you can use with `--params-table` and `--row-index`.

In [4]:
# Define the sweep values
sigma = noise["params"]["sigma"]
theta_c = find_theta_c_from_degree_distribution(
    degree_distribution=degree_distribution,
    sigma=sigma,
    theta_bracket=(1e-6, 1.0),
)

theta_values = np.concatenate([
    np.linspace(0.5 * theta_c, theta_c, 5),
    np.linspace(theta_c, 2 * theta_c, 10)[1:],
])

# Build a table with dot-path columns that override the base config
records = []
for theta_idx, theta in enumerate(theta_values, start=1):
    run_id = f"theta_{theta_idx:02d}"
    records.append({
        "run_id": run_id,
        "system.params.theta": float(theta),
    })

df = pd.DataFrame(records)


In [6]:
# Output path for the sweep table
path_folder = "params"
name_sweep_table = "sweep_theta_double_well_configuration_model_poisson.tsv"
sweep_path = repo_root / path_folder / name_sweep_table
sweep_path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(sweep_path, sep="	", index=False)
print(f"Wrote sweep table to {sweep_path}")


Wrote sweep table to /Users/niccolo/Desktop/Dynamics_Networks/params/sweep_theta_double_well_configuration_model_poisson.tsv
